Instalando Pacote

In [1]:
!git clone https://github.com/huggingface/transformers;
!cd transformers; pip3 install .

Cloning into 'transformers'...
remote: Enumerating objects: 166858, done.
remote: Counting objects: 100% (74737/74737), done.
remote: Compressing objects: 100% (4253/4253), done.
remote: Total 166858 (delta 72175), reused 70538 (delta 70469), pack-reused 92121
Receiving objects: 100% (166858/166858), 163.52 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (127584/127584), done.
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7921011 sha256=e86036d8f4602e5667122c53

In [2]:
! pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
! pip install datasets

Puxando Dataset de Discurso de Ódio

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer



# link: https://huggingface.co/datasets/ag_news

In [6]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/t5_key_text/original/base_offensive_test.csv', sep = ';', decimal = ',')
test.head()

,text,label
0,#ibelieveblaseyford is liar she is fat ugly li...,1
1,@user @user @user I got in a pretty deep debat...,0
2,"...if you want more shootings and more death, ...",0
3,Angels now have 6 runs. Five of them have come...,0
4,#Travel #Movies and Unix #Fortune combined Vi...,0


In [7]:
# X_train = dataset['train']['text']
X_test = test['text']
# y_train = dataset['train']['label']
y_test = test['label']

In [8]:
nclasses = len(np.unique(y_test))

In [9]:
nclasses

2

In [10]:
def puxa_dados_normal(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [11]:
def puxa_original(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')

  X = csv['texto'].values
  y = csv['label'].values


  return X, y

In [12]:
def puxa_dados_normal_pkl(caminho):
  csv = pd.read_pickle(caminho)
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [13]:
def puxa_dados_lambada(caminho, original):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = pd.read_csv(original, sep = ';', decimal = ',')

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [14]:
def puxa_dados_lambada_filtro(caminho, original, limiar):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = pd.read_csv(original, sep = ';', decimal = ',')

  csv = csv[csv['prob_text'] >= limiar].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [15]:
def puxa_dados_lambada_pkl_filtro(caminho, original, limiar):
  csv = pd.read_pickle(caminho)
  original = pd.read_csv(original, sep = ';', decimal = ',')

  csv = csv[csv['prob_text'] >= limiar].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [16]:
def desempenho_LSTM(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [17]:
def desempenho_CNN(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Conv1D(filters=100, kernel_size = 3, activation = 'relu',
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),

        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu',
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [18]:
# Evaluate the model
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')


In [19]:
def desempenho_transformers(X_train, X_test, y_train, y_test,classes, model,type_model, epochs, batch, max_len):
  from simpletransformers.classification import ClassificationModel
  import pandas as pd
  import logging
  import sklearn

  train_df = pd.DataFrame(X_train)
  train_df['target'] = y_train

  eval_df = pd.DataFrame(X_test)
  eval_df['target'] = y_test



  # Create a ClassificationModel
  model = ClassificationModel(type_model, model, num_labels=classes, args={'num_train_epochs':epochs,
                                                                           'train_batch_size':batch,
                                                                           'max_seq_length':max_len,
                                                                          #  "fp16":False,
                                                                           'overwrite_output_dir': True})

  # Train the model
  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score,f1=f1_multiclass)

  return result['acc'], result['f1']





In [20]:

y_test = np.array(y_test)


# Temperatura

In [22]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/temperatura/base_offensive_temperatura_001.csv',
      '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/temperatura/base_offensive_temperatura_01.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/temperatura/base_offensive_temperatura_025.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/temperatura/base_offensive_temperatura_05.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/temperatura/base_offensive_temperatura_075.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/temperatura/base_offensive_temperatura_09.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/temperatura/base_offensive_temperatura_099.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       ]


roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])


  roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  print('Roberta')
  print('ACC:' + str(roberta[0]))
  print('F1:' + str(roberta[1]))


  roberta_acc.append(roberta[0])
  roberta_f1.append(roberta[1])

0


<ipython-input-13-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1020 [00:00<?, ?it/s]

...

Running Epoch 3 of 4:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Roberta
ACC:0.7651162790697674
F1:0.7088732328276768


In [23]:
data = {'temperatura': [ 0.01,0.1,0.25,0.5,0.75,0.9,0.99], # 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
        'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,}



base_resultados = pd.DataFrame.from_dict(data)

In [24]:
base_resultados

,temperatura,ROBERTA_ACC,ROBERTA_F1
0,0.01,0.702326,0.588714
1,0.10,0.686047,0.553956
2,0.25,0.801163,0.743776
3,0.50,0.740698,0.681407
4,0.75,0.769767,0.721508
5,0.90,0.612791,0.596065
6,0.99,0.765116,0.708873


# Tamanho

In [25]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_10.csv',
      '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_15.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_20.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_25.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_30.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_35.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_40.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_45.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_50.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       ]


roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])


  roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  print('Roberta')
  print('ACC:' + str(roberta[0]))
  print('F1:' + str(roberta[1]))


  roberta_acc.append(roberta[0])
  roberta_f1.append(roberta[1])

0


<ipython-input-13-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1020 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/32 [00:00<?, ?it/s]

...

Running Epoch 3 of 4:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Roberta
ACC:0.7616279069767442
F1:0.7241480744346445


In [26]:
data = {'tamanho': [ 10,15,20,25,30,35,40,45,50], # 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
        'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,}



base_resultados = pd.DataFrame.from_dict(data)

In [27]:
base_resultados

,tamanho,ROBERTA_ACC,ROBERTA_F1
0,10,0.622093,0.603338
1,15,0.752326,0.720796
2,20,0.684884,0.642861
3,25,0.680233,0.662142
4,30,0.775581,0.673421
5,35,0.697674,0.646211
6,40,0.796512,0.746820
7,45,0.811628,0.762860
8,50,0.761628,0.724148


# Número f

In [28]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/numero_f/base_offensive_numero_f_1.csv',
      '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/numero_f/base_offensive_numero_f_3.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/numero_f/base_offensive_numero_f_5.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/numero_f/base_offensive_numero_f_10.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/numero_f/base_offensive_numero_f_15.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/numero_f/base_offensive_numero_f_20.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(1)+'.csv',
       ]


roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])


  roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  print('Roberta')
  print('ACC:' + str(roberta[0]))
  print('F1:' + str(roberta[1]))


  roberta_acc.append(roberta[0])
  roberta_f1.append(roberta[1])

0


<ipython-input-13-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1020 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/32 [00:00<?, ?it/s]

...

Running Epoch 3 of 4:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Roberta
ACC:0.7813953488372093
F1:0.7317405010784803


In [29]:
data = {'numero_f': [ 1,3,5,10,15,20], # 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
        'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,}



base_resultados = pd.DataFrame.from_dict(data)

In [30]:
base_resultados

,numero_f,ROBERTA_ACC,ROBERTA_F1
0,1,0.655814,0.637759
1,3,0.804651,0.773258
2,5,0.758140,0.719833
3,10,0.684884,0.658985
4,15,0.796512,0.747466
5,20,0.781395,0.731741
